<a href="https://colab.research.google.com/github/BruceYoungcct/voice/blob/main/iot_devp1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3>IoT開発実習（５）</h3>

Amazon EC2の環境でMQTT メッセージを送受信する

<h3>一、Git、Node.js をインストールして、AWS CLI を設定する</h3>

（１）Git をインストールする


Amazon EC2 Instance Connect ウィンドウで、次のコマンドを使用してインスタンスを更新する。

In [ ]:
sudo yum update -y

Amazon EC2 Instance Connectウィンドウで、次のコマンドを使用して Git をインストールする。


In [ ]:
sudo yum install git -y

Gitのインストールを確認する

In [ ]:
git --version

     <補足：>
     yumは「Yellowdog Updater Modified」、 apt-getのaptは「Advanced Package Tool」
     両者の違いはディストリビューションの違いこと
     yum は RedHat系のOSで使用します。EC2のLinux環境はRedHat系のOS
     apt-get は DebianやUbuntuのOSで使用

（２）Node.js をインストールする

Amazon EC2 Instance Connect ウィンドウで、次のコマンドを使用してノードバージョンマネージャー (Node Version Manager , 略称nvm) をインストールする。

In [ ]:
curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.34.0/install.sh | bash

    <解説> 
    nvmを使うとひとつのシステム内に 複数のバージョンの Node.js をインストールしておき、必要に応じてバージョンを切り替えて使えるようになる。現在利用しているNode.jｓのバージョンを確認するには、コマンド” nvm current ”を使用する。

Amazon EC2 Instance Connect ウィンドウで、このコマンドを使用して nvm を有効にする。

In [ ]:
. ~/.nvm/nvm.sh

Amazon EC2 Instance Connect ウィンドウで、このコマンドを使用して、nvm を使用し、最新バージョンの Node.js をインストールする。

In [ ]:
nvm install --lts

Node.js がインストールされ、正しく実行されていることをテストする

In [ ]:
node -e "console.log('Running Node.js ' + process.version)"

（３）AWS environment variables（AWS環境変数）をセットアップする

![image.png](attachment:image.png)

（４）AWS CLI （AWS Command Line Interface）を設定する

Amazon EC2 Instance Connect ウィンドウで、次のコマンドを入力する。

In [ ]:
aws configure

    <解説> 　
    aws configureは、AWS Command Line Interface（CLI）を使用してAWSアカウントの認証情報を設定するためのコマンドである。
    このコマンドを使用することで、AWS CLIを介してAWSサービスにアクセスするためのアクセスキー、シークレットアクセスキー、デフォルトのリージョン、および出力形式を設定することができる。

In [ ]:
AWS Access Key ID [None]: “AWS Access Key ID”
Secret Access Key [None]: “AWS Secret Access Key”
Default region name [None]: ap-northeast-1
Default output format [None]: json

次のコマンドを使用して、AWS CLI の設定をテストし、エンドポイントが表示されたら、AWS CLIの設定が完了

In [ ]:
aws iot describe-endpoint --endpoint-type iot:Data-ATS

    <補足>AWS IoT Coreのエンドポイント

    ①AWS IoT Core - コントロールプレーンエンドポイント
    AWS IoT Coreサービスへの管理操作やコントロールに使用されるエンドポイント

In [ ]:
構造：<prefix>.iot.<region>.amazonaws.com

    ・<prefix>：AWSアカウントごとに一意の識別子で、固有のエンドポイントという。一般的にはxxxxxxxxxxxxのようなランダムな文字列
    ・このエンドポイントを使用して、AWS IoT Coreサービスに対してデバイスの登録、ルールの設定、トピックの管理などの操作を行うことができる。

    ②AWS IoT Core – データプレーンエンドポイント</br>
    デバイスからのデータ送信やAWS IoT Coreからのデータ受信に使用されるエンドポイント</br>


In [ ]:
構造：<account-specific-prefix>.iot.<region>.amazonaws.com</br>


    ・<account-specific-prefix>：AWSのアカウントの固有のエンドポイントにエンドポイントタイプATSを指定する。一般的にはxxxxxxxxxxxx-atsのようなランダムな文字列
    ・ATSは、Authenticating Things Serviceの略称、デバイスがデータを送信する際には、ATSプロトコルを使用してデータプレーンエンドポイントに接続し、データの送信や認証を行う。ATSはTLS（Transport Layer Security）プロトコルを使用してデータの暗号化とセキュアな通信を提供する。
    ・このエンドポイントを使用して、デバイスからのデータ送信やAWS IoT Coreへのデータ受信を行うことができる。

    <補足>
    ①AWS IoT Core - コントロールプレーンエンドポイントを出力するコマンド

In [ ]:
aws iot describe-endpoint

    <補足>
    ②AWS IoT Core – データプレーンエンドポイント


In [ ]:
aws iot describe-endpoint --endpoint-type iot:Data-ATS

<h3>二、仮想デバイス用の AWS IoT リソースを作成する</h3>

（１）Linux インスタンスで AWS IoT モノのオブジェクトを作成する

Amazon EC2 Instance Connect ウィンドウで、次のコマンドを使用してインスタンスを更新する。

In [ ]:
aws iot create-thing --thing-name "MyIotThing"

JSON レスポンスは以下のようになる。

In [ ]:
{
    "thingName": "MyIotThing",
    "thingArn": "arn:aws:iot:ap-northeast-1:615570136058:thing/MyIotThing",
    "thingId": "1464ac1e-850d-4543-8f59-5bfe2f3ccc2d"
}


（２）Linux インスタンスで 作成したAWS IoT モノのオブジェクトを確認する</br>
（１）で作成したAWS IoTモノのオブジェクトを確認するには、以下の手順：</br>
AWS IoT Coreにアクセスし、左側のメニューバーから「管理」→「すべてのデバイス」→「モノ」




![image.png](attachment:image.png)

（３）Linux インスタンスで AWS IoT キーと証明書を作成する

Amazon EC2 Instance Connectウィンドウで、証明書とキーファイルを保存するディレクトリを作成する。

In [ ]:
mkdir ~/cert

Amazon EC2 Instance Connect ウィンドウで、このコマンドを使用し、Amazon 認証機関 (CA) 証明書のコピーをダウンロードする。

In [ ]:
curl -o ~/certs/Amazon-root-CA-1.pem \
        https://www.amazontrust.com/repository/AmazonRootCA1.pem 


Amazon EC2 Instance Connect ウィンドウで、次のコマンドを実行し、プライベートキー、パブリックキー、および X.509 証明書ファイルを作成する。また、このコマンドは、AWS IoT に証明書を登録して有効化する。

In [ ]:
aws iot create-keys-and-certificate \
    --set-as-active \
    --certificate-pem-outfile "~/certs/device.pem.crt" \
    --public-key-outfile "~/certs/public.pem.key" \
    --private-key-outfile "~/certs/private.pem.key"

    <補足>
    キーと証明書の役割

    （１）プライベートキー（Private Key）:
        デバイスのプライベートキーは、デバイス自体が所有する秘密鍵である。
        デバイスがメッセージを暗号化したり、デジタル署名を作成したりする際に使用される。
        プライベートキーはデバイス内で安全に保管される必要がある。

    （２）パブリックキー（Public Key）:
        デバイスのパブリックキーは、デバイスのプライベートキーから生成される公開鍵である。
        デバイスがメッセージを暗号化したり、他のデバイスからのメッセージを復号化したりする際に使用される。
        パブリックキーは他のデバイスやクラウドサービスに公開される。

    （３）X.509証明書ファイル（デバイス証明書）:
        X.509証明書は、デバイスの認証に使用されるデジタル証明書である。
        証明書にはデバイスのパブリックキーとデバイスの識別情報が含まれている。
        デバイスがAWS IoT Coreに接続する際に、証明書を使用してデバイスの正当性を確認する。


（４）証明書を保存する

レスポンスは次のようになる。

In [ ]:
{
    "certificateArn": "arn:aws:iot:ap-northeast-1:615570136058:cert/fcff16a7050d0769d9be850e3818b1b4f7ead533610cea6c8a0df5f1e91abb9e",
    "certificateId": "38abdcc7385ae85a8d85539edfd456b667198746883c7700756cb2f650e75197",
    "certificatePem": "-----BEGIN CERTIFICATE-----\nMIIDWjCCAkKgAwIBAgIVAK9yc4uZNV+jqQHBSJr1ZPRzl31GMA0GCSqGSIb3DQEB\nCwUAME0xSzBJBgNVBAsM…
.....

certificateArn を保存し、後続のコマンドで使用できるようにする。

    <補足>
    certificateArnは、AWS IoT Coreで使用される証明書のAmazon Resource Name（ARN）である。
    AWS IoT Coreでは、デバイスやアプリケーションが使用する証明書にもARNが割り当てられる。   
    certificateArnは、特定の証明書を識別するための一意の識別子であり、AWSリソースの管理やアクセス制御などの操作に使用される。
    以下のような形式で表される:

In [ ]:
arn:aws:iot: <リージョン>:<アカウント番号>:cert/<証明書ID>